In [1]:
import os
import cv2
import torch
import wandb
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from torch.utils.data import DataLoader
import albumentations as A
from datasets import GTA5, CityScapes
from models.deeplabv2.deeplabv2 import get_deeplab_v2
from models.bisenet.build_bisenet import BiSeNet
from train import train
from utils import (save_results, plot_loss, plot_mIoU, plot_IoU)
from train import train_dacs

# Load environment variables
load_dotenv()
api_key = os.getenv('WANDB_API_KEY')
wandb.login(key=api_key)

# Set random seed for reproducibility
torch.cuda.manual_seed(42)

# Mapping from class IDs to labels
id_to_label = {
    0: 'road', 1: 'sidewalk', 2: 'building', 3: 'wall', 4: 'fence',
    5: 'pole', 6: 'light', 7: 'sign', 8: 'vegetation', 9: 'terrain',
    10: 'sky', 11: 'person', 12: 'rider', 13: 'car', 14: 'truck',
    15: 'bus', 16: 'train', 17: 'motorcycle', 18: 'bicycle', 255: 'unlabeled'
}

BATCH_SIZE = 6
NC=19
NUM_WORKERS = 8
cityscape_size = (256,512)
GTA5_size = (256,512)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ardaerendogru (aeddea). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/arda/.netrc


In [2]:
transforms = A.Compose([
    A.Resize(cityscape_size[0], cityscape_size[1]),
])
transforms_GTA5 = A.Compose([
    A.Resize(GTA5_size[0], GTA5_size[1]),
])

cityscapes_train_dataset = CityScapes('./Cityscapes', 'train', transform=transforms)
cityscapes_test_dataset = CityScapes('./Cityscapes', 'val', transform=transforms)
GTA5_dataset = GTA5('./GTA5', transform=transforms_GTA5)

cityscapes_train_dataloader = DataLoader(cityscapes_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
GTA5_dataloader = DataLoader(GTA5_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

len(cityscapes_train_dataloader), len(cityscapes_test_dataloader), len(GTA5_dataloader)


(262, 84, 417)

## 2.1 DeepLabV2

In [3]:

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
init_lr = 2.5e-4
model_deeplab = get_deeplab_v2(num_classes=19, pretrain=True, pretrain_model_path='./models/deeplab_resnet_pretrained_imagenet.pth').cuda()
optimizer_deeplab = torch.optim.Adam(model_deeplab.parameters(), lr=init_lr)

deeplab_result = train( model_deeplab,
                        loss_fn, optimizer_deeplab,
                        cityscapes_train_dataloader,
                        cityscapes_test_dataloader, 
                        5,
                        'step2_1_DeepLabV2')
save_results(model_deeplab, deeplab_result, "deeplab_performance_metrics_2_1", height=cityscape_size[0], width=cityscape_size[1], iterations=10)
plot_loss(deeplab_result, "DeepLabV2", "step2_1", "CityScapes", "CityScapes")
plot_mIoU(deeplab_result, "DeepLabV2", "step2_1", "CityScapes", "CityScapes")
plot_IoU(deeplab_result, "DeepLabV2", "step2_1", "CityScapes", "CityScapes")

torch.save(model_deeplab.state_dict(), "./checkpoints/deeplab_model_2_1.pth")
# model_deeplab.load_state_dict(torch.load("./checkpoints/deeplab_model.pth"))
# model_deeplab.eval()


KeyboardInterrupt: 

## 2.2 BiseNet


In [ ]:
model_bisenet = BiSeNet(19, 'resnet18').cuda()

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
init_lr = 2.5e-4 #0.0001
optimizer_bisenet = torch.optim.Adam(model_bisenet.parameters(), lr=init_lr)
bisenet_result_2_2 = train(model_bisenet, loss_fn, optimizer_bisenet,cityscapes_train_dataloader,cityscapes_test_dataloader, 5,'step2_BiseNet')
save_results(model_bisenet, bisenet_result_2_2, "bisenet_performance_metrics_2_2", height=cityscape_size[0], width=cityscape_size[1], iterations=10)

plot_loss(bisenet_result_2_2, "BiSeNet", "step2_2", "CityScapes", "CityScapes")
plot_mIoU(bisenet_result_2_2, "BiSeNet", "step2_2", "CityScapes", "CityScapes")
plot_IoU(bisenet_result_2_2, "BiSeNet", "step2_2", "CityScapes", "CityScapes")

torch.save(model_bisenet.state_dict(), "./checkpoints/bisenet_model_2_2.pth")


## 3.1 Bisenet

In [ ]:

model_bisenet_3_1 = BiSeNet(19, 'resnet18').cuda()
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
init_lr = 2.5e-4 #0.0001
optimizer_bisenet_3_1 = torch.optim.Adam(model_bisenet_3_1.parameters(), lr=init_lr)
bisenet_result_3_1 = train(model_bisenet_3_1, loss_fn, optimizer_bisenet_3_1,GTA5_dataloader,cityscapes_test_dataloader, 5,'step3_1_BiseNet')
save_results(model_bisenet_3_1, bisenet_result_3_1, "bisenet_performance_metrics_3_1", height=cityscape_size[0], width=cityscape_size[1], iterations=10)
plot_loss(bisenet_result_3_1, "BiSeNet", "step3_1", "GTA5", "CityScapes")
plot_mIoU(bisenet_result_3_1, "BiSeNet", "step3_1", "GTA5", "CityScapes")
plot_IoU(bisenet_result_3_1, "BiSeNet", "step3_1", "GTA5", "CityScapes")

torch.save(model_bisenet_3_1.state_dict(), "./checkpoints/bisenet_model_3_1.pth")
# model_bisenet_3_1.load_state_dict(torch.load("./checkpoints/bisenet_model_3_1.pth"))
# model_bisenet_3_1.eval()


## 3.2 Bisenet with augmentation



In [ ]:

augmentations = {
    'transform1': A.Compose([
        A.Resize(GTA5_size[0],GTA5_size[1]),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
    ]),
    'transform2': A.Compose([
        A.Resize(GTA5_size[0],GTA5_size[1]),
        A.HueSaturationValue(p=0.5),
        A.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 1), p=0.5),
    ]),
    'transform3': A.Compose([
        A.Resize(GTA5_size[0],GTA5_size[1]),
        A.HorizontalFlip(p=0.5),
        A.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 1), p=0.5),
        A.GaussNoise(p=0.5),
    ]),
    'transform4': A.Compose([
        A.Resize(GTA5_size[0],GTA5_size[1]),
        A.HorizontalFlip(p=0.5),
        A.HueSaturationValue(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
    ]),
    'transform5': A.Compose([
        A.Resize(GTA5_size[0],GTA5_size[1]),
        A.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 1), p=0.5),
        A.GaussNoise(p=0.5),
    ]),
    'transform6': A.Compose([
        A.Resize(GTA5_size[0],GTA5_size[1]),
        A.HorizontalFlip(p=0.5),
        A.GaussNoise(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(p=0.5),
    ])
}

best_score = 0
best = ''
for key, value in augmentations.items():
    cityscapes_train_dataset = CityScapes('./Cityscapes', 'train', transform = value)
    cityscapes_test_dataset = CityScapes('./Cityscapes', 'val', transform = value)
    GTA5_dataset = GTA5('./GTA5', transform = value)


    cityscapes_train_dataloader = DataLoader(cityscapes_train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    GTA5_dataloader = DataLoader(GTA5_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    
    model_bisenet_3_2 = BiSeNet(19, 'resnet18').cuda()
    init_lr = 2.5e-4 #0.0001
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
    optimizer = torch.optim.Adam(model_bisenet_3_2.parameters(), lr=init_lr)

    bisenet_result3_2 = train(model_bisenet_3_2, loss_fn, optimizer,GTA5_dataloader,cityscapes_test_dataloader, 5,'step3_BiseNet')
    
        
    save_results(model_bisenet_3_2, bisenet_result3_2, f"bisenet_performance_metrics_3_2_{key}", height=cityscape_size[0], width=cityscape_size[1], iterations=10)
    plot_loss(bisenet_result3_2, "BiSeNet", f"step3_2_{key}", "GTA5", "CityScapes")
    plot_mIoU(bisenet_result3_2, "BiSeNet", f"step3_2_{key}", "GTA5", "CityScapes")
    plot_IoU(bisenet_result3_2, "BiSeNet", f"step3_2_{key}", "GTA5", "CityScapes")
    torch.save(model_bisenet_3_2.state_dict(), f"./checkpoints/bisenet_model_3_2_{key}.pth")
    if best_score < bisenet_result3_2[3][-1]:
        best_score = bisenet_result3_2[3][-1]
        best = key
best        

# 4.1 FDA

In [ ]:
GTA5_dataset = GTA5('./GTA5', transform=augmentations[best], FDA = 0.09)
GTA5_dataloader = DataLoader(GTA5_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
model_bisenet_4_1_FDA = BiSeNet(19, 'resnet18').cuda()
init_lr = 2.5e-4 #0.0001
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.Adam(model_bisenet_4_1_FDA.parameters(), lr=init_lr)

bisenet_result4_1_FDA = train(model_bisenet_4_1_FDA, loss_fn, optimizer,GTA5_dataloader,cityscapes_test_dataloader, 5,'step3_BiseNet')

    
save_results(model_bisenet_4_1_FDA, bisenet_result4_1_FDA, "bisenet_performance_metrics_4_1_FDA", height=cityscape_size[0], width=cityscape_size[1], iterations=10)
plot_loss(bisenet_result4_1_FDA, "BiSeNet", "step4_1_FDA", "GTA5", "CityScapes")
plot_mIoU(bisenet_result4_1_FDA, "BiSeNet", "step4_1_FDA", "GTA5", "CityScapes")
plot_IoU(bisenet_result4_1_FDA, "BiSeNet", "step4_1_FDA", "GTA5", "CityScapes")
torch.save(model_bisenet_4_1_FDA.state_dict(), f"./checkpoints/bisenet_model_4_1_FDA.pth")


# 4.2 DACS

1- In the paper they say that they are using pretrained model. but in pseudocode they say that they are initializing the model with random parameters.


In [3]:
# GTA5_dataset = GTA5('./GTA5', transform=augmentations[best], FDA = 0.09)
GTA5_dataset = GTA5('./GTA5', transform=transforms_GTA5, FDA = 0.09)

GTA5_dataloader = DataLoader(GTA5_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
model_bisenet_4_2_DACS = BiSeNet(19, 'resnet18').cuda()
init_lr = 2.5e-4 #0.0001
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.Adam(model_bisenet_4_2_DACS.parameters(), lr=init_lr)

bisenet_result4_2_DACS = train_dacs(model_bisenet_4_2_DACS, loss_fn, optimizer,GTA5_dataloader, cityscapes_train_dataloader,cityscapes_test_dataloader, 5,'step4_2_DACS', 20)
save_results(model_bisenet_4_2_DACS, bisenet_result4_2_DACS, "bisenet_performance_metrics_4_2_DACS", height=cityscape_size[0], width=cityscape_size[1], iterations=10)
plot_loss(bisenet_result4_2_DACS, "BiSeNet", "step4_2_DACS", "GTA5 + CityScapes (DACS)", "CityScapes")
plot_mIoU(bisenet_result4_2_DACS, "BiSeNet", "step4_2_DACS", "GTA5 + CityScapes (DACS)", "CityScapes")
plot_IoU(bisenet_result4_2_DACS, "BiSeNet", "step4_2_DACS", "GTA5 + CityScapes (DACS)", "CityScapes")

torch.save(model_bisenet_4_2_DACS.state_dict(), "./checkpoints/bisenet_model_4_2_DACS.pth")


  0%|          | 0/5 [00:00<?, ?it/s]

Step: 0.0
Step: 0.023980815347721823
Step: 0.047961630695443645
Step: 0.07194244604316546
Step: 0.09592326139088729
Step: 0.11990407673860912
Step: 0.14388489208633093
Step: 0.16786570743405277
Step: 0.19184652278177458
Step: 0.2158273381294964
Step: 0.23980815347721823
Step: 0.2637889688249401
Step: 0.28776978417266186
Step: 0.3117505995203837
Step: 0.33573141486810554
Step: 0.3597122302158273
Step: 0.38369304556354916
Step: 0.407673860911271
Step: 0.4316546762589928
Step: 0.4556354916067146
Step: 0.47961630695443647
Step: 0.5035971223021583
Step: 0.5275779376498801
Step: 0.5515587529976019
Step: 0.5755395683453237
Step: 0.5995203836930456
Step: 0.6235011990407674
Step: 0.6474820143884892
Step: 0.6714628297362111
Step: 0.6954436450839329
Step: 0.7194244604316546
Step: 0.7434052757793765
Step: 0.7673860911270983
Step: 0.7913669064748201
Step: 0.815347721822542
Step: 0.8393285371702638
Step: 0.8633093525179856
Step: 0.8872901678657075
Step: 0.9112709832134293
Step: 0.935251798561151
Ste

In [4]:
import torch

# Example tensor with shape [6, 1, 64, 64]
mask = torch.randn(6, 1, 64, 64)

# Squeeze the tensor to remove the dimension of size 1
squeezed_mask = mask.squeeze(1)  # Specify dimension to ensure only the intended dimension is removed

print(squeezed_mask.shape)  # Should print torch.Size([6, 64, 64])

torch.Size([6, 64, 64])
